とりあえずjndiという文字列が入ったものを全期間にて取り出しを行った後、
JNDILookUpに該当する文字列、具体的には

- $jndi
- ${jndi
- {jndi
- %25%7Bjndi
- %25jndi
- %7Bjndi
とあるものを取り出す。

In [1]:
import json
from ElasticSearcher import search
from Grapher import visualizeTimeData

query = {
    "wildcard": {
        "request": "*jndi*"
    }
}
r = search(query=query, starting=1911, ending=2112)
with open('resultData/hit_JNDI.json', 'w') as f:
    json.dump(r, f, indent=4)

c:\users\shion\pycharmprojects\projectroute112021\venv\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


以下の調査で判明したこと
- 当該ハニーポットでは、CVE-2021-44228は2021/12/9以前は検出されなかった。
-

In [9]:
from matplotlib import pyplot as plt

r1 = []
hits = ['{jndi', '%25jndi', "%7bjndi"]
for d in r:
    x = d['_source']['request'].lower()
    for h in hits:
        if h in x:
            r1.append(d)
            break
print(len(r1))
print(len(r))
import pandas as pd

t = pd.to_datetime("2021/12/09 12:00:00").timestamp()
b = False
for d in r1:
    p_time = pd.to_datetime(d['_source']['@timestamp']).timestamp()
    if p_time < t:
        b = True
        break
if not b: print("no CVE-2021-44228 found before 2021/12/9")
ending = pd.to_datetime("2021/12/31 00:00:00")
plt.rcParams["figure.figsize"] = [8, 6]
visualizeTimeData(r1, starting=t, ending=ending.timestamp(), title='CVE-2021-44228 全データの期間分布',
                  loc="Log4J調査", period=12 * 60 * 60, x_axis='hour')
starting = pd.to_datetime("2021/12/09 12:00:00")
ending = pd.to_datetime("2021/12/10 00:00:00")
visualizeTimeData(r1, starting=t, ending=ending.timestamp(), title='CVE-2021-44228 初期動向',
                  loc="Log4J調査", period=1 * 30 * 60, x_axis='hour')
initialData = []
ending = pd.to_datetime("2021/12/10 12:00:00")
for d in r1:
    p_time = pd.to_datetime(d['_source']['@timestamp']).timestamp()
    if p_time < ending.timestamp():
        print(d['_source']['source_ip'])
        print(d['_source']['geoip']['city_name'] if 'city_name' in d['_source']['geoip'] else '')

9699
11467
no CVE-2021-44228 found before 2021/12/9
185.220.101.136

194.48.199.78

185.220.101.180

185.220.101.180

185.220.100.253

194.48.199.78

45.90.142.58
Tokyo
185.220.100.245

185.220.101.38

209.141.60.19
Las Vegas
171.25.193.78

185.56.80.65

194.48.199.78

213.95.149.22
Arberg
204.8.156.142
Boston
194.48.199.78

194.48.199.78

185.220.101.32

45.90.142.58
Tokyo
194.48.199.78

171.25.193.20

45.90.142.58
Tokyo
45.90.142.58
Tokyo
107.189.1.178
Roost
185.220.101.160

185.220.100.251

185.220.100.254

185.220.100.255

185.38.175.132

107.189.1.160
Roost
171.25.193.25

185.220.101.147

45.90.142.58
Tokyo
185.220.100.253

185.220.101.146

45.90.142.58
Tokyo
185.220.101.147

194.48.199.78

194.48.199.78



ポート番号を見てみる

In [3]:
from DataGrouper import groupByPorts
r_ports=groupByPorts(r1)
r_ports=sorted(r_ports.items(),key=lambda x:len(x[1]),reverse=True)
print(len(r_ports))
for port, g in r_ports:
    print("%s : %d" % (port,len(g)))

1029
80 : 1261
8080 : 1258
8983 : 415
5480 : 306
8000 : 270
8081 : 243
8085 : 211
102 : 178
9200 : 171
8089 : 155
7547 : 148
8009 : 142
8088 : 127
81 : 118
8010 : 92
8087 : 72
8086 : 69
8084 : 63
8888 : 62
8083 : 61
9092 : 60
5984 : 53
8090 : 52
8834 : 51
1400 : 50
16992 : 49
9090 : 46
7548 : 46
1433 : 45
3689 : 41
8181 : 40
8008 : 40
33338 : 39
1194 : 39
4040 : 38
18080 : 36
7001 : 36
8500 : 36
2001 : 33
13579 : 31
7100 : 31
443 : 31
27017 : 30
5000 : 30
4567 : 29
50100 : 29
1024 : 29
7077 : 28
49152 : 27
50880 : 27
8001 : 27
7000 : 27
995 : 25
3128 : 25
8545 : 25
9002 : 25
7549 : 24
9001 : 24
5901 : 24
992 : 23
7010 : 23
9000 : 22
21 : 22
9944 : 22
6060 : 22
49153 : 21
1311 : 21
65535 : 21
8880 : 20
9595 : 20
8060 : 20
554 : 20
3000 : 19
2004 : 19
8899 : 19
2379 : 18
110 : 18
4001 : 18
88 : 18
8761 : 18
60000 : 18
666 : 17
8554 : 17
7777 : 17
8820 : 17
10000 : 17
5431 : 17
9990 : 17
9060 : 17
2455 : 16
51106 : 16
64738 : 16
7678 : 16
3780 : 16
5555 : 16
22 : 15
2082 : 15
9042 : 15
33